In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tqdm
from bpe_tokenizer import BPETokenizer
from transformers import PreTrainedTokenizerFast
from tfidf_vectorizer import CustomTfidfVectorizer
from ensemble import EnsembleClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
import transformers
print("transformers version:", transformers.__version__)

transformers version: 4.36.2


In [3]:
df = pd.read_csv('/Users/mahdiislam/Higher Studies/MAIA Study/Semester 1/Software Engineering/Project/Artificial_text_detection/Dataset/train_v2_drcat_02.csv')
train_ex = pd.read_csv('/Users/mahdiislam/Higher Studies/MAIA Study/Semester 1/Software Engineering/Project/Artificial_text_detection/Dataset/train_essays.csv')

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size = 0.2, random_state=202)
bpe = BPETokenizer(30522).train(train_ex)
seq_length = 15000
tokenizer = bpe.get_fast_tokenizer(seq_length)


In [6]:
tokenized_texts_test = []

for text in X_test.tolist():
    tokenized_texts_test.append(tokenizer.tokenize(text))

tokenized_texts_train = []

for text in X_train.tolist():
    tokenized_texts_train.append(tokenizer.tokenize(text))

tokenized_texts_train_ex = []    
for text in train_ex['text'].tolist():
    tokenized_texts_train_ex.append(tokenizer.tokenize(text))

Token indices sequence length is longer than the specified maximum sequence length for this model (19032 > 15000). Running this sequence through the model will result in indexing errors


In [7]:
tfidf_vectorizer = CustomTfidfVectorizer()
tfidf_vectorizer.fit(tokenized_texts_train_ex)
tf_train = tfidf_vectorizer.fit_transform(tokenized_texts_train)
tf_test = tfidf_vectorizer.transform(tokenized_texts_test)

In [1]:
# Instantiate the EnsembleClassifier
ensemble_classifier = EnsembleClassifier()

# Fit the model
ensemble_classifier.fit(tf_train, y_train)

# Make predictions
predictions = ensemble_classifier.predict(tf_test)

cm = confusion_matrix(y_test, predictions)
accuracy = accuracy_score(y_test, predictions)
labels = ('Not artificially generated', 'Artificially Generated')
sns.heatmap(cm, annot=True, xticklabels=labels, yticklabels=labels)
plt.show()

NameError: name 'EnsembleClassifier' is not defined